# Example 2: Eliptical slice sampling (ESS) for prior-normalized posterior 

In [721]:
# To start this notebook with more than one thread run "export JULIA_NUM_THREADS=4" in the terminal 
# before starting the jupyter notebook

# Ensure that Julia was launched with an appropriate number of threads
println(Threads.nthreads())

6


In [722]:
# Import module. 
using Revise
using PriorNormalization

In [723]:
# Import packages 
using AdaptiveMCMC # for using adaptive MCMC sampling
using ApproxFun # for approximating gammainvccdf by a Chbychev interpolant  
using CairoMakie # for plots 
using Dates # to measure computational time 
using Distributions 
using FlexiMaps # for log-range  
using ForwardDiff # for AD
using JLD2 # for saving and loading results
using LinearAlgebra # to represent the identity matrix as "I" 
using MCMCChains
using Random # for generating random noise 
using StatsBase # for defining customized distributions 
using StatsFuns # for defining customized distributions 
using StatsPlots # for plotting 
using SparseArrays # for efficient storing of the forward operator 
using SpecialFunctions
using Turing # for setting up the model and sampling 
using Optim # for ML and MAP estimation 
using Plots: Plots, plot, plot!, scatter, scatter!, savefig, surface, surface!
using ProgressMeter # to show progress 

In [724]:
# Prepare arguments.  
nr_chains = 6 # number of chains to sample 
nr_samples_raw = 10^5 # number of samples 
thin = 10^1 # Thinning factor; only every thin-th sample is stored
init = "prior" # Initialization option: "MAP", "prior"
nr_samples = Int64( nr_samples_raw/thin )
step_size = 0.02 # pCN step size 

# Tests for step_size = 0.01: 
# nr_samples_raw = 10^4, thin = 10^0: 3s, 37% & 5s, 58%
# nr_samples_raw = 10^5, thin = 10^1: 24s, 37% & 36s, 39%
# nr_samples_raw = 10^6, thin = 10^2: 

# Tests for step_size = 0.02: 
# nr_samples_raw = 10^4, thin = 10^0: 5s, 16% & 5s, 27%
# nr_samples_raw = 10^5, thin = 10^1: 28s, 15% & 23s, 17%
# nr_samples_raw = 10^6, thin = 10^2: 

0.02

## Signal deblurring problem: Define the data model 

In [725]:
# Model parameters 
σ² = 0.03^2 # noise variance 
kernel_width = 0.02 # width of the Gaussian kernel
N_dense = 1_000 # number of points for the dense model
N_coarse = 128 # number of points for the coarse model
tt = [0.17, 0.39, 0.48, 0.73, 0.83] # Positions of the increments
dx = [1, -2.4, 2.8, -0.6, -0.8] # Values of the increments

5-element Vector{Float64}:
  1.0
 -2.4
  2.8
 -0.6
 -0.8

In [726]:
# Define the piecewise constant signal. 
function signal(t; tt=[0.17, 0.39, 0.48, 0.73, 0.83], 
    dx = [1, -2.4, 2.8, -0.6, -0.8])
    x = 0
    Ij = findall(x -> x < t, tt)
    if !isempty(Ij)
        x = sum(dx[Ij])
    end
    
    return x
end

signal (generic function with 1 method)

In [727]:
# Set up the dense data model  

# Generate the dense grid (we assume that the signal vanishes at t=0)
t_dense = (1:N_dense) / N_dense 

# Generate the dense forward operator 
S = reshape(repeat(t_dense, N_dense, 1), N_dense, N_dense)
T = S'
F_dense = (6.4/N_dense) * exp.(-1/(2*kernel_width^2) * (T.-S).^2) 

# Generate the dense step signal and observations 
x_dense = signal.(t_dense) # signal values 
y_dense = F_dense * x_dense # observations

1000-element Vector{Any}:
  3.74278937539227e-18
  5.744144243534786e-18
  8.79393927412621e-18
  1.3429811599511571e-17
  2.0459022030414613e-17
  3.1090554911693094e-17
  4.713037498942281e-17
  7.126926866908596e-17
  1.0750603997755944e-16
  1.6176805186960793e-16
  2.428187113838113e-16
  3.6358103971191115e-16
  5.430629429581705e-16
  ⋮
  2.653109939010615e-16
  1.693119370541826e-16
  1.0509846120628357e-16
  6.233097419208321e-17
  3.399747075949146e-17
  1.535468383152762e-17
  3.2021048354653674e-18
 -4.614439835542763e-18
 -9.541335133345305e-18
 -1.2547439067780209e-17
 -1.4280276203172293e-17
 -1.5171704985465577e-17

In [728]:
# Set up the coarse data model  

# Generate the dense grid (we assume that the signal vanishes at t=0)
t_coarse = (1:N_coarse) / N_coarse 

# Generate the dense forward operator 
S = reshape(repeat(t_coarse, N_coarse, 1), N_coarse, N_coarse)
T = S'
F_coarse = (6.4/N_coarse) * exp.(-1/(2*kernel_width^2) * (T.-S).^2) 

# Get the coarse grid and forward operator 
stride = 6 # use every stride-th point 
t_obs = t_coarse[1:stride:end]
F_coarse = F_coarse[1:stride:end, :]

# Find the nearest points in the dense grid
m = length(t_obs)
I_dense = zeros(Int, m)
for j in 1:m
    I_dense[j] = argmin(abs.(t_dense .- t_obs[j]))
end
     
# Coarse data with added noise
Random.seed!(123) # Setting the seed 
y_coarse = y_dense[I_dense] .+ sqrt(σ²)*randn(m);

In [729]:
# Invertible finite difference matrix
aux = ones(N_coarse) * [-1.0, 1.0]'
L = spzeros(Float64,N_coarse,N_coarse)
L[2:end,:] = spdiagm(0=>-1*ones(N_coarse), 1=>ones(N_coarse))[1:N_coarse-1,1:N_coarse]
L[1,1] = 1 

# Change coordinates to promot sparsity in z = Lx
FL = F_coarse / L # F_coarse * inv(L)

# Whitening
FL_w = (1/sqrt(σ²)) * FL
y_w = (1/sqrt(σ²)) * y_coarse

# Rename varables for simplicity 
F = FL_w 
y = y_w 
M, N = size(F)

(22, 128)

## Select the model: $r=-1$

In [730]:
# Select hyper-hyper-parameters 
model_nr = 4 

# Parameter of of generalized gamma hyper-prior 
r_range = [ 1.0, .5, -.5, -1.0 ]; 
β_range = [ 1.501, 3.0918, 2.0165, 1.0017 ];
ϑ_range = [ 5*10^(-2), 5.9323*10^(-3), 1.2583*10^(-3), 1.2308*10^(-4) ];

# Select hyper-hyper-parameters 
r = r_range[model_nr] # power parameter 
β = β_range[model_nr] # shape parameter 
ϑ = ϑ_range[model_nr] # scale parameter

0.00012308

In [731]:
interval = -5..5
# Create an array of functions
if r > 0 
    Γinvccdf_cheb = Fun(τ -> gammainvccdf(β, 1, 0.5*erfc(τ/sqrt(2)) ), interval)
else 
    Γinvccdf_cheb = Fun(τ -> gammainvccdf(β, 1, 0.5 + 0.5*erf(τ/sqrt(2)) ), interval)
end

# Calculate the value and derivative at the boundaries
Γinvccdf_val_left = Γinvccdf_cheb(interval.left)
Γinvccdf_val_right = Γinvccdf_cheb(interval.right)

Γinvccdf_deriv_left = ForwardDiff.derivative(Γinvccdf_cheb, interval.left)
Γinvccdf_deriv_right = ForwardDiff.derivative(Γinvccdf_cheb, interval.right)

# Define the extended function
function Γinvccdf_cheb_extd(τ)
    if τ < interval.left
        return Γinvccdf_val_left + Γinvccdf_deriv_left * (τ - interval.left)
    elseif τ > interval.right
        return abs(Γinvccdf_val_right + Γinvccdf_deriv_right * (τ - interval.right))
    else
        return Γinvccdf_cheb(τ)
    end
end

Γinvccdf_cheb_extd (generic function with 1 method)

In [732]:
# Define the posterior density 
function logpdf_likelihood(τ, u; F, y, r, β, ϑ, Φ::Function) 

    # Get z-value by using transport map 
    z = similar(u)
    z = priorNormalizing_KR_inv_tu_fast( u, τ; r, β, ϑ, Φ )
    
    # Get log-posterior 
    logpdf = -0.5*norm(F*z-y)^2
    
    return logpdf
end

logpdf_likelihood (generic function with 1 method)

In [733]:
log_likelihood(ξ) = -logpdf_likelihood(
    ξ[1:2:end-1], ξ[2:2:end]; 
    F, y, r, β, ϑ, Φ=Γinvccdf_cheb_extd
)

log_likelihood (generic function with 1 method)

In [734]:
# Load the MAP estimates 

# MAP estimate of the prior-normalized posterior 
@load "data/deblurring_model$(model_nr)_MAP_priorNormalized.jld2" τ_MAP u_MAP
# Initialize an empty vector to store the interleaved values
priorNormalized_MAP = Vector{Float64}(undef, 2*N)
# Interleave τ_MAP and u_MAP
priorNormalized_MAP[1:2:end] .= τ_MAP;
priorNormalized_MAP[2:2:end] .= u_MAP;

In [735]:
## Generate random samples from the standard normal prior 
priorNormalized_prior = Array{Float64}(undef, 2*N, nr_samples)
τ_prior_samples = rand(Normal(0,1), N, nr_chains)
u_prior_samples = rand(Normal(0,1), N, nr_chains)

for j in 1:nr_chains
    priorNormalized_prior[1:2:end,j] .= τ_prior_samples[:,j]
    priorNormalized_prior[2:2:end,j] .= u_prior_samples[:,j]
end

In [736]:
# Choose an initialization for the MCMC chains 
init_param = Array{Float64}(undef, 2*N, nr_chains)

# Use MAP estimate 
if init=="MAP"
    # Select the initial set of parameters 
    for j in 1:nr_chains 
        init_param[:,j] = priorNormalized_MAP[:]
    end

    # Select the file names for saving the later MCMC results  
    filename_priorNormalized = joinpath(
        "data", 
        "deblurring_model$(model_nr)_mcmc_initMAP_pCN_priorNormalized_samples$(nr_samples_raw)_thin$(thin)_chains$(nr_chains).jld2"
    )

# Use random prior samples 
elseif init=="prior"
    # Select the initial set of parameters 
    for j in 1:nr_chains 
        init_param[:,j] = priorNormalized_prior[:,j]
    end

    # Select the file names for saving the later MCMC results 
    filename_priorNormalized = joinpath(
        "data", 
        "deblurring_model$(model_nr)_mcmc_initPrior_pCN_priorNormalized_samples$(nr_samples_raw)_thin$(thin)_chains$(nr_chains).jld2"
    )

# Throw an error if none of the available options is provided
else
    error("Invalid initialization option provided: $init. Please choose either 'MAP' or 'prior'.")
end

"data/deblurring_model4_mcmc_initPrior_pCN_priorNormalized_samples100000_thin10_chains6.jld2"

## pCN sampling 

In [737]:
# Define the logarithm of the acceptance probability of the pCN algorithm
function log_acc_prob(τ_old, u_old, τ_prop, u_prop) 
    N = length(τ_old)

    ξ_old = zeros(2*N) 
    ξ_old[1:2:end-1] = τ_old
    ξ_old[2:2:end] = u_old
    
    ξ_prop = zeros(2*N) 
    ξ_prop[1:2:end-1] = τ_prop
    ξ_prop[2:2:end] = u_prop
    
    log_α = min(0, log_likelihood(ξ_old) - log_likelihood(ξ_prop) )
    
    return log_α
end

log_acc_prob (generic function with 1 method)

In [738]:
# Define the Gibbs sampling function
function pCN_sampler(τ_init, u_init, step_size, iterations)
    N = length(τ_init)
    step_size_param = sqrt( 1 - step_size^2 )
    acc_counter = 0
    
    # Preallocate space to store samples
    samples_τ = zeros(iterations, N)
    samples_u = zeros(iterations, N)

    # Initialize values 
    τ_old = τ_init
    u_old = u_init
    τ_prop = zeros(N)
    u_prop = zeros(N)
    
    # Store samples
    samples_τ[1, :] = τ_old
    samples_u[1, :] = u_old
    
    for i in 2:iterations
        
        # Step 1: Proposal 
        v1 = rand(Normal(0,1), N)
        τ_prop = step_size_param*τ_old + step_size*v1
        v2 = rand(Normal(0,1), N)
        u_prop = step_size_param*u_old + step_size*v2
        
        # Step 2: MH accept/reject
        z = rand(1)
        log_α = log_acc_prob(τ_old, u_old, τ_prop, u_prop)
        if log(z[1]) <= minimum([0, log_α])
            τ_old = τ_prop
            u_old = u_prop 
            acc_counter += 1
        end
            
        # Store samples
        samples_τ[i, :] = τ_old
        samples_u[i, :] = u_old
    end

    acc_rate = acc_counter/(iterations-1)
    
    return samples_τ, samples_u, acc_rate
end

pCN_sampler (generic function with 1 method)

In [739]:
# Initialize
nr_parameters = 2*N
Tau_init = zeros(Float64, N, nr_chains)
U_init = zeros(Float64, N, nr_chains)
Tau = zeros(Float64, nr_samples_raw, N, nr_chains)
U = zeros(Float64, nr_samples_raw, N, nr_chains)
acc_rates = zeros(Float64, nr_chains)
chn_values = zeros(Float64, nr_samples, 2*N, nr_chains)

# Start the wall clock timer
wall_start = now()

# Use multiple threads. 
@showprogress Threads.@threads for j in 1:nr_chains   
    
    # Initilization 
    Tau_init[:,j] = init_param[1:2:end-1,j]
    U_init[:,j] = init_param[2:2:end,j]
    
    # Generate samples
    Tau[:,:,j], U[:,:,j], acc_rates[j] = pCN_sampler(
        Tau_init[:,j], 
        U_init[:,j], 
        step_size, 
        nr_samples_raw
    )
    
    # Store values 
    chn_values[:,1:2:end-1,j] = Tau[1:thin:nr_samples_raw,:,j]
    chn_values[:,2:2:end,j] = U[1:thin:nr_samples_raw,:,j]
end

# End the wall clock timer
wall_end = now()
wall_duration_ms = wall_end - wall_start
# Convert wall duration to seconds
wall_duration_priorNormalized = Dates.value(wall_duration_ms) / 1000

# Define the parameter names (τ[1], u[1], τ[2], u[2], ...)
param_names_τ = [string("τ[", i, "]") for i in 1:N ]
param_names_u = [string("u[", i, "]") for i in 1:N ]

# Interleave θ and z names
param_names = Vector{String}(undef, nr_parameters)
param_names[1:2:end-1] .= param_names_τ
param_names[2:2:end] .= param_names_u

# Create the Chains object
chn_priorNormalized = Chains(chn_values, Symbol.(param_names));

# Free the memory occupied by chn_values
chn_values = nothing
GC.gc() # Optionally trigger garbage collection manually

Progress: 100%|█████████████████████████████████████████| Time: 0:00:23


In [740]:
mean_acc_rate = mean(acc_rates)

println(wall_duration_priorNormalized) 
println(mean_acc_rate) 
println(acc_rates)

23.909
0.1758634253009197
[0.18977189771897718, 0.18172181721817218, 0.15973159731597317, 0.18051180511805118, 0.15302153021530215, 0.1904219042190422]


In [741]:
# Save the MCMC chain and computational time
@save filename_priorNormalized chn_priorNormalized wall_duration_priorNormalized mean_acc_rate

# Multivariate potential scale reduction factor (MPSRF) 
# To check convergence: Should be below 1.1

#gelmandiag_multivariate(chn_priorNormalized)